In [7]:
import pandas as pd
import numpy as np
import pickle
from selenium import webdriver

In [23]:
driver = webdriver.Chrome("C:/Users/ansem/Desktop/chromedriver_win32/chromedriver.exe")

### headless
# options = webdriver.ChromeOptions()
# options.add_argument('headless')
# driver = webdriver.Chrome('chromedriver.exe', options=options)

C:\Users\ansem\AppData\Local\Temp/ipykernel_22216/551986738.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:/Users/ansem/Desktop/chromedriver_win32/chromedriver.exe")


In [24]:
driver.get('https://movie.naver.com/')
driver.find_element("xpath", '/html/body/div/div[3]/div/div[1]/div/div/ul/li[3]/a').click()

In [25]:
driver.find_element("xpath", '//*[@id="old_content"]/table/tbody/tr[6]/td[2]/div/a').click()
driver.find_element("xpath", '/html/body/div/div[4]/div[1]/a').click()

In [27]:
# 영화정보
movie_title = driver.find_element("xpath", '/html/body/div/div[4]/div[3]/div[1]/div[2]/div[1]/h3/a[1]').text
genre = driver.find_element("xpath", '//*[@id="content"]/div[1]/div[2]/div[1]/dl/dd[1]/p/span[1]/a').text
attendance = driver.find_element("xpath", '//*[@id="content"]/div[1]/div[2]/div[1]/dl/dd[5]/div/p').text
print(movie_title,'|', genre,'|',attendance)

헤어질 결심 | 멜로/로맨스 | 1,831,555명(08.17 기준)


In [28]:
# 관람객 연령대
attendance_age = []
attendance_per = []
for i in range(5):
    attendance_per.append(driver.find_element("xpath", f'//*[@id="content"]/div[1]/div[2]/div[1]/div[2]/div/div[2]/div[{i+1}]/strong[1]').text)
    attendance_age.append(driver.find_element("xpath", f'//*[@id="content"]/div[1]/div[2]/div[1]/div[2]/div/div[2]/div[{i+1}]/strong[2]').text)
attendance_info = dict(zip(attendance_age,attendance_per))
print('관람객 연령대')
print(attendance_info)

관람객 연령대
{'10대': '4%', '20대': '40%', '30대': '35%', '40대': '14%', '50대\n이상': '6%'}


In [29]:
driver.find_element("xpath", '//*[@id="movieEndTabMenu"]/li[5]/a/em').click()

In [30]:
# 연령별 평점
attendance_rate = []
for i in range(5):
    attendance_rate.append(driver.find_element("xpath", f'html/body/div/div[4]/div[2]/div[1]/div[4]/div[1]/div[1]/div[2]/div[1]/div[2]/div/div/div[1]/div[2]/div[{i+1}]/strong[2]').text)
attendance_rate_info = dict(zip(attendance_age,attendance_rate))
print('연령별 평점')
print(attendance_rate_info)

연령별 평점
{'10대': '8.95', '20대': '8.45', '30대': '8.45', '40대': '8.70', '50대\n이상': '8.72'}


In [31]:
# 관람 포인트
point_field = []
point_per = []
for i in range(5):
    point_field.append(driver.find_element("xpath", f'//*[@id="netizen_point_graph"]/div/div[2]/ul/li[{i+1}]/strong').text)
    point_per.append(driver.find_element("xpath", f'//*[@id="netizen_point_graph"]/div/div[2]/ul/li[{i+1}]/span').text)

point_view = dict(zip(point_field,point_per))
print('관람 포인트')
print(point_view)

관람 포인트
{'연출': '26%', '연기': '24%', '스토리': '18%', '영상미': '21%', 'OST': '11%'}


In [32]:
driver.switch_to.frame('pointAfterListIframe')

In [33]:
# 관람객 평점
review_df = pd.DataFrame(columns=['평점','리뷰','공감','비공감'])

for j in range(10):
    for i in range(10):
        grade = driver.find_element("xpath", f'/html/body/div/div/div[5]/ul/li[{i+1}]/div[1]/em').text
        review = driver.find_element("xpath", f'//*[@id="_filtered_ment_{i}"]').text
        good = driver.find_element("xpath", f'/html/body/div/div/div[5]/ul/li[{i+1}]/div[3]/a[1]/strong').text
        bad = driver.find_element("xpath", f'/html/body/div/div/div[5]/ul/li[{i+1}]/div[3]/a[2]/strong').text
        review_df.loc[10*j+i] = [grade,review,good,bad]
    driver.find_element("xpath", f'//*[@id="pagerTagAnchor{j+1}"]').click()

review_df

,평점,리뷰,공감,비공감
0,10,네게 미결로 남고 싶은 내 삶의 흔적,2819,240
1,7,"사랑에는 결심이 필요하지 않죠, 자연스러운 거니까. 그런데 헤어질 땐 결심이 필요하죠.",2064,166
2,10,이 영화는 4시 전후로 보는 것이 좋다. 엔딩크레딧이 올라가고 밖으로 나와 어둑해진...,2114,254
3,10,"잠겨 죽어도 좋으니, 너는 물처럼 내게 밀려오라",1907,185
4,10,의심하고 미행하고 끝내 수갑 채우려 한다는 점에서 사랑은 하나의 수사극과 같다. 그...,1487,123
...,...,...,...,...
95,10,한번봐선 아쉬움.. 두세번봐야 파악할수있는 진심..,66,2
96,9,붕괴해야 사랑이 시작되는 그 슬픈 모순,73,9
97,7,박찬욱이 생각하는 사랑이 이런거구나,64,5
98,10,어느 시점부터 서래가 해일을 사랑하게 됐을까? 진심은 사람을 움직이게 만드는 힘이 있다.,76,17


In [34]:
driver.switch_to.default_content()

In [35]:
expert_rate_avg = ''
for i in range(4):
    expert_rate_avg += driver.find_element("xpath", f'/html/body/div/div[4]/div[2]/div[1]/div[4]/div[4]/div/div[1]/div/em[{i+1}]').get_attribute("innerHTML")
    expert_num = driver.find_element("xpath", f'//*[@id="content"]/div[1]/div[4]/div[4]/div/div[1]/span/em').get_attribute("innerHTML")

print(f'{expert_num}명 참여 | 평균 평점 : {expert_rate_avg}')


14 명 참여 | 평균 평점 : 8.71


In [36]:
# 전문가
review_exp_df = pd.DataFrame(columns=['저널리스트','평점','리뷰'])
for i in range(100):
    try:
        name=driver.find_element("xpath", f'//*[@id="content"]/div[1]/div[4]/div[4]/div/div[2]/ul/li[{i+1}]/div[1]/dl/dt/a').get_attribute("innerHTML")
        grade=driver.find_element("xpath", f'//*[@id="content"]/div[1]/div[4]/div[4]/div/div[2]/ul/li[{i+1}]/div[2]/div/div/em').get_attribute("innerHTML")
        review=driver.find_element("xpath", f'//*[@id="content"]/div[1]/div[4]/div[4]/div/div[2]/ul/li[{i+1}]/p').get_attribute("innerHTML")
        review_exp_df.loc[i] = [name,grade,review]
    except: pass
index = len(review_exp_df)

for i in range(100):
    try:
        name=driver.find_element("xpath", f'//*[@id="content"]/div[1]/div[4]/div[4]/div/div[3]/div/ul/li[{i+1}]/div[2]/dl/dd').get_attribute("innerHTML").strip('\n').strip('\t').strip('\n').strip('| ')
        grade=driver.find_element("xpath", f'//*[@id="content"]/div[1]/div[4]/div[4]/div/div[3]/div/ul/li[{i+1}]/div[1]/em').get_attribute("innerHTML")
        review=driver.find_element("xpath", f'//*[@id="content"]/div[1]/div[4]/div[4]/div/div[3]/div/ul/li[{i+1}]/div[2]/p').get_attribute("innerHTML")
        review_exp_df.loc[index+i] = [name,grade,review]
    except: pass
        

review_exp_df

,저널리스트,평점,리뷰
0,이은선,9,박찬욱 감독은 이 영화로 지금 시점에서 자신이 추구하는 어떤 경지를 목격하게 한다....
1,이지혜,8,&lt;헤어질 결심&gt;은 자신만의 스타일이 있는 창작자가 만든 순도 높은 결과물...
2,정시우,9,"의심은 관심이 되고, 잠복근무는 관음의 알리바이가 되고, 대화는 시(운율)가 되고,..."
3,김소미,9,박찬욱 영화의 정서적 만조
4,김철홍,7,미결되어 영원 재생될 영화. 새롭진 않지만 마침내
5,남선우,9,"침투하는 디스플레이와 침잠하는 데이터 너머, 끝내 안개를 껴안는 영화"
6,배동미,8,한국영화계의 영원한 모더니스트 박찬욱
7,송경원,9,카메라의 권능과 변태적 편집의 위력. 숨소리마저 예쁜 극한의 세공
8,이용철,10,마.침.내. 이 엔딩에 도착하려고 그 오랜 시간 영화에 빠진 것처럼
9,이우빈,9,감정이 사건을 이끄는 영화. 그 반대가 아니라


In [38]:
review_exp_df.iloc[0,2]

'박찬욱 감독은 이 영화로 지금 시점에서 자신이 추구하는 어떤 경지를 목격하게 한다. 중요한 건 고고한 예술가의 도취적 결과가 아니라 대중예술로서의 겸허한 손짓이라는 점이다. 보는 이로 하여금 인물들의 말을 곱씹고 영화의 미학에 나의 시선을 덧대보고 싶은 욕구를 가지게 한다는 점에서 &lt;헤어질 결심&gt;은 훌륭하다. 어쩌면 이 영화에 분석적 접근은 필요하지 않을지 모른다. 정취의 파도를 하릴없이 얻어맞은 우리 각자가 그저 무너지고 깨어진 채로 존재하는 것이 이 영화를 최선으로 욕망하고 품는 방식일 것이기에. 애수와 회한 사이를 기품 있게 오가는 탕웨이와 박해일의 연기는 흠잡을 곳이 없다. 올해는 이미 그들의 해다.'

In [96]:
# pickle로 저장
with open('review_df.pickle', 'wb') as f:
    pickle.dump(review_df, f)

with open('review_exp_df.pickle', 'wb') as f:
    pickle.dump(review_exp_df, f)